In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohanrajakash","key":"934fb39f285cb7adb53a95ca6d8ad988"}'}

In [2]:
# Make directory and move the kaggle.json file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
 98% 2.25G/2.29G [00:16<00:00, 222MB/s]
100% 2.29G/2.29G [00:16<00:00, 150MB/s]


In [3]:
!unzip -q chest-xray-pneumonia.zip

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
import os

# Define the paths to your data directories
train_dir = 'chest_xray/train'
test_dir = 'chest_xray/test'

In [5]:
# Define image dimensions and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Create the training dataset from the directory
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Create the testing dataset from the directory
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='binary',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.


In [6]:
# Load the pre-trained ResNet50 model without its top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model so they are not re-trained
base_model.trainable = False

# Add our new custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x) # This layer flattens the features
# Add the final prediction layer for our binary (Normal/Pneumonia) classification
predictions = Dense(1, activation='sigmoid')(x)

# Create the final model by combining the base and our new layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with an optimizer, loss function, and metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# You can print a summary of the model architecture
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,589,761 (89.99 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
# Train the model for 10 epochs (passes through the data)
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset
)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 54s 251ms/step - accuracy: 0.8359 - loss: 0.3712 - val_accuracy: 0.7821 - val_loss: 0.4593
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 70s 216ms/step - accuracy: 0.9574 - loss: 0.1153 - val_accuracy: 0.8301 - val_loss: 0.4000
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 36s 218ms/step - accuracy: 0.9692 - loss: 0.0913 - val_accuracy: 0.8381 - val_loss: 0.4024
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 40s 211ms/step - accuracy: 0.9697 - loss: 0.0766 - val_accuracy: 0.8221 - val_loss: 0.4615
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 43s 225ms/step - accuracy: 0.9758 - loss: 0.0682 - val_accuracy: 0.7692 - val_loss: 0.6486
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 38s 208ms/step - accuracy: 0.9756 - loss: 0.0620 - val_accuracy: 0.8301 - val_loss: 0.4588
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 40s 205ms/step - accuracy: 0.9775 - loss: 0.0550 - val_accuracy: 0.8317 - val_loss: 0.4448
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 36s 220ms/step - accuracy: 0.9791 - loss: 0